
## 20200615_00
とりあえずEfficient net にそのまま突っ込んで実験  
-> データ数の偏りから、全部 bining に分類

In [ ]:
import chainer
from chainer.dataset import convert

import melanoma

title = "20200615_00"

device = 0
batch_size = 32
width = 224
height = 224
epoch = 10
n_classes = 2
output_dir = melanoma.constants.RESULT_DIR / "20200615_00"
transfered_weights = None

model = melanoma.models.EfficientNet()
loss_func = model.loss_func
optimizer = chainer.optimizers.Adam()
predictor = melanoma.predictor.Predictor(model, (width, height))

if device >= 0:
    model.to_gpu(device)
    predictor.to_gpu(device)

optimizer.setup(model)

train_ds, val_ds, test_ds = melanoma.dataset.DatasetBuilder(img_size=(width, height), n_classes=n_classes).build()

train_itr = chainer.iterators.MultiprocessIterator(train_ds, batch_size, repeat=True)
val_itr = chainer.iterators.MultiprocessIterator(val_ds, batch_size, repeat=False)
test_itr = chainer.iterators.MultiprocessIterator(test_ds, batch_size, repeat=False)

updater = chainer.training.StandardUpdater(
    train_itr, optimizer, converter=convert.concat_examples, loss_func=loss_func, device=device
)
evaluator = chainer.training.extensions.Evaluator(
    val_itr, model, device=device, eval_func=loss_func
)
trainer = melanoma.trainer.TrainerBuilder(updater, epoch ,evaluator, output_dir).build()


In [ ]:
# run train
trainer.run()


In [ ]:
# evaluation
chainer.serializers.load_npz(melanoma.constants.RESULT_DIR / title / "snapshot_main_model_{}.npz".format(epoch), model)
melanoma.utility.start_pdb(
    lambda : melanoma.evaluate.evaluate(predictor, 
                           test_itr,
                           [l.name for l in melanoma.constants.Labels],
                           output_dir / f"{title}_eval",
                           device=device,
                          )
)